# pip installing libraries

In [2]:
!pip install transformers
!pip install datasets
!pip install pynvml
!pip install evaluate
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 27.0 MB/s eta 0:00:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Utils for GPU

In [4]:
# ===========================================
# ||                                       ||
# ||       Section 1: Importing modules    ||
# ||                                       ||
# ===========================================
import torch
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo


# ===========================================
# ||                                       ||
# ||  Section 2: utils functions for gpu   ||
# ||             and device                ||
# ||                                       ||
# ===========================================

def check_gpu_availability():
    # Check if CUDA is available
    print(f"Cuda is available: {torch.cuda.is_available()}")


def getting_device(gpu_prefence=True) -> torch.device:
    """
    This function gets the torch device to be used for computations,
    based on the GPU preference specified by the user.
    """

    # If GPU is preferred and available, set device to CUDA
    if gpu_prefence and torch.cuda.is_available():
        device = torch.device('cuda')
    # If GPU is not preferred or not available, set device to CPU
    else:
        device = torch.device("cpu")

    # Print the selected device
    print(f"Selected device: {device}")

    # Return the device
    return device


# Define a function to print GPU memory utilization
def print_gpu_utilization():
    # Initialize the PyNVML library
    nvmlInit()
    # Get a handle to the first GPU in the system
    handle = nvmlDeviceGetHandleByIndex(0)
    # Get information about the memory usage on the GPU
    info = nvmlDeviceGetMemoryInfo(handle)
    # Print the GPU memory usage in MB
    print(f"GPU memory occupied: {info.used // 1024 ** 2} MB.")


# Define a function to print training summary information
def print_summary(result):
    # Print the total training time in seconds
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    # Print the number of training samples processed per second
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    # Print the GPU memory utilization
    print_gpu_utilization()

def clean_gpu():
    # Get current GPU memory usage
    print("BEFORE CLEANING:")
    print(f"Allocated: {cuda.memory_allocated() / 1024 ** 3:.2f} GB")
    print(f"Cached: {cuda.memory_cached() / 1024 ** 3:.2f} GB")
    print("\n")
    # Free up PyTorch and CUDA memory
    torch.cuda.empty_cache()
    cuda.empty_cache()

    # Run garbage collection to free up other memory
    gc.collect()

    # Get new GPU memory usage
    print("AFTER CLEANING:")
    print(f"Allocated: {cuda.memory_allocated() / 1024 ** 3:.2f} GB")
    print(f"Cached: {cuda.memory_cached() / 1024 ** 3:.2f} GB")

# Import libraries

In [5]:


import transformers
from datasets import load_dataset, load_metric,  Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import os
import nltk
import torch
import evaluate
import sys
import pandas as pd
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import os
import torch.cuda as cuda
import gc
import optuna
#from utils4gpu import *

# Utils for Model

In [6]:

# getting the max length of the tokenized tweet
def getting_max_length(tokenizer, items):

  # initialize a list for lengths
  max_len = 0

  # iterate over the list
  for item in items:

    # record lenght of each item
    lenght_of_item = len(tokenizer.encode(item, add_special_tokens = True))

    if lenght_of_item > max_len:

      max_len = lenght_of_item

  return max_len


# Checking GPU, selecting model and device

In [7]:
# CHECK IF GPU IS UP
check_gpu_availability()

# SAVE THE DEVICE WE ARE WORKING WITH
device = getting_device(gpu_prefence=True)

# SHOULD BE FEW MB
print_gpu_utilization()

# SETTING HF CHECKPOINT/MODEL
model_nm = "distilbert-base-uncased"

Cuda is available: True
Selected device: cuda
GPU memory occupied: 261 MB.


# Defining dataframes

In [8]:
# Read csv files to create pandas dataframes
path2test = '/content/drive/MyDrive/GOOD DATA/clean_test_data.csv'
test_df = pd.read_csv(path2test)

path2val = '/content/drive/MyDrive/GOOD DATA/clean_validation_data.csv'
validation_df = pd.read_csv(path2val)

path2train = '/content/drive/MyDrive/GOOD DATA/augmented_cleaned_train_df.csv'
train_df = pd.read_csv(path2train)

# Renaming columns
train_df.rename(columns = {"target":"labels"}, inplace = True)
validation_df.rename(columns = {"target":"labels"}, inplace = True)
test_df.rename(columns = {"target":"labels"}, inplace = True)

# pandas2dataset
ds_train = Dataset.from_pandas(train_df)
ds_validation = Dataset.from_pandas(validation_df)
ds_test = Dataset.from_pandas(test_df)

# Tokenization, tensorization and collider

In [9]:
# ===========================================
# ||                                       ||
# ||Section 5: tokenization, tensorization ||
# ||              and collider             ||
# ||                                       ||
# ===========================================

# IMPORTING OUR TOKENIZER
tokz = AutoTokenizer.from_pretrained(model_nm)

# GETTING THE LENGHT MAX
max_len = getting_max_length(tokz,ds_train["text"])

# DEFINING A TOKENIZE FUNCTION TO TOKENIZE BOTH THE TWO DATASETS
def tok_func(x): return tokz(x["text"], truncation=True, padding = "max_length", max_length=max_len)

# TOKENIZING THE DS
tok_ds_train = ds_train.map(tok_func, batched=True, remove_columns=['text','id', 'keyword', 'location'])
tok_ds_validation = ds_validation.map(tok_func, batched=True, remove_columns=['text','id', 'keyword', 'location'])
tok_ds_test = ds_test.map(tok_func, batched=True, remove_columns=['text','id', 'keyword', 'location'])

# CREATE A DATASET TO FEED THE MODEL
ds = DatasetDict({"train":tok_ds_train,
                  "validation":tok_ds_validation,
             "test": tok_ds_test})

# GETTING THE COLLATOR
data_collator = DataCollatorWithPadding(tokenizer=tokz)

Map:   0%|          | 0/4872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

# Metrics

In [10]:
# 1) F1 and ACCURACY

# now that we have our callable object, we define a function that the trainer can use to compute its metric => we cannot call directly metrics.compute because the output
# of the model is not a prediction but a logist
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

# Hyperparameter tuning

In [11]:

num_r = 1

# Define the search space for hyperparameters using Optuna's distributions.
def objective(trial):
    global num_r

    # Rename folder containing the old model and files
    if os.path.exists("/content/distilbert-base-uncased"):
      os.rename("/content/distilbert-base-uncased", os.path.join(os.path.dirname("/content/distilbert-base-uncased"), str(num_r)))
    num_r += 1

    # IMPORTING THE MODEL
    model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels = 2).to(device)
    # checking if the model is on the gpu
    print_gpu_utilization()

    # Hyperparameters to optimize
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-4)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
    num_train_epochs = trial.suggest_int('num_train_epochs', 1, 5)

    # setting the hyperparameter for the trainer
    training_args = TrainingArguments(
        model_nm,
        evaluation_strategy = "epoch",
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        logging_steps = 50, # FROM BELOW MEMORY TRICKS
        gradient_accumulation_steps=16, # adding them to offset small batch size due to memory problem => so 2*8 => 16 batch-size traning
        fp16 = True
    )

    # passing in the hyperparameter for the trainer
    trainer = Trainer(
        model = model, # our model
        args = training_args, # hyperparameter defined before
        train_dataset = ds["train"],
        eval_dataset = ds["validation"],
        compute_metrics = compute_metrics, # evaluation function defined before
        data_collator = data_collator,
    )

    # TRAINING LOOP
    print(" ")
    print("START TRAINING")
    print(" ")
    trainer.train()
    print("DONE TRAINING")

    # TESTING
    print(" ")
    print("START VALIDATION")
    print(" ")
    predictions = trainer.predict(ds["validation"])
    eval_result = compute_metrics(predictions)
    print(eval_result)
    print("DONE VALIDATION")

    # Return the evaluation metric to be optimized by Optuna.
    return 1 - eval_result['f1']

# Define the Optuna study and run the hyperparameter search.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)
print(study.best_params)
best_trial = study.best_trial

# Train the model with the best hyperparameters found by optuna and evaluate it on the test data.
best_num_train_epochs = best_trial.params['num_train_epochs']
best_weight_decay = best_trial.params['weight_decay']
best_learning_rate = best_trial.params['learning_rate']


[I 2023-03-30 09:07:42,891] A new study created in memory with name: no-name-ec2158fb-06f0-4244-9bfb-aadd64a741ad


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

GPU memory occupied: 1355 MB.
 
START TRAINING
 


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.616907,0.753695,0.736131
1,0.641300,0.547528,0.778325,0.763674
2,0.532300,0.512546,0.789819,0.777782
3,0.483400,0.508817,0.788998,0.776208


DONE TRAINING
 
START VALIDATION
 


...

GPU memory occupied: 2663 MB.
 
START TRAINING
 


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.683320,0.651067,0.649891
1,0.687300,0.677819,0.669951,0.649460


DONE TRAINING
 
START VALIDATION
 


[I 2023-03-30 09:42:44,628] Trial 19 finished with value: 0.3505402037304647 and parameters: {'learning_rate': 1.048633392020499e-06, 'weight_decay': 3.3029056905903962e-06, 'num_train_epochs': 2}. Best is trial 7 with value: 0.16094418197254912.


{'accuracy': 0.6699507389162561, 'f1': 0.6494597962695353}
DONE VALIDATION
{'learning_rate': 2.6675414031743722e-05, 'weight_decay': 2.7516452256746474e-06, 'num_train_epochs': 4}


# Model and Hyperparams



In [12]:

# Rename folder containing the old model
if os.path.exists("/content/distilbert-base-uncased"):
    os.rename("/content/distilbert-base-uncased", os.path.join(os.path.dirname("/content/distilbert-base-uncased"), '1'))

# IMPORTING THE MODEL
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels = 2).to(device)
# checking if the model is on the gpu
print_gpu_utilization()

# setting the hyperparameter for the trainer
training_args = TrainingArguments(
    model_nm,
    evaluation_strategy = "epoch",
    learning_rate=best_learning_rate,
    weight_decay=best_weight_decay,
    num_train_epochs=best_num_train_epochs,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps = 50, # FROM BELOW MEMORY TRICKS
    gradient_accumulation_steps=16, # adding them to offset small batch size due to memory problem => so 2*8 => 16 batch-size traning
    fp16 = True
    )

# passing in the hyperparameter for the trainer
trainer = Trainer(
    model = model, # our model
    args = training_args, # hyperparameter defined before
    train_dataset = ds["train"],
    eval_dataset = ds["validation"],
    compute_metrics = compute_metrics, # evaluation function defined before
    data_collator = data_collator,
)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

GPU memory occupied: 2663 MB.


# Training and testing the model

In [13]:
# TRAINING LOOP
print(" ")
print("START TRAINING ")
print(" ")
trainer.train()
print("DONE TRAINING")

# TESTING
print(" ")
print("\033[32mSTARTING FINAL TESTING OF THE MODEL\033[0m")
print(" ")
predictions = trainer.predict(ds["test"])
eval_result = compute_metrics(predictions)
print(eval_result)
print("DONE TESTING")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


 
START TRAINING 
 


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.416596,0.819376,0.818156
1,0.484500,0.390525,0.833333,0.831909
2,0.364200,0.393761,0.843186,0.841510
3,0.311900,0.394396,0.840722,0.839056


DONE TRAINING
 
STARTING FINAL TESTING OF THE MODEL
 


{'accuracy': 0.8305975049244911, 'f1': 0.8295095463285094}
DONE TESTING


# Variation and bias

In [14]:
from datasets import concatenate_datasets
from sklearn.model_selection import KFold

ds = concatenate_datasets([tok_ds_train, tok_ds_validation, tok_ds_test])

n=5
kf = KFold(n_splits=n, random_state=42, shuffle=True)

accuracy = []
f1 = []
set1 = train_df
set1.rename(columns = {"target":"labels"}, inplace = True)
i = 0
for train_index, val_index in kf.split(set1):
  i+=231
  if os.path.exists("/content/distilbert-base-uncased"):
     os.rename("/content/distilbert-base-uncased", os.path.join(os.path.dirname("/content/distilbert-base-uncased"), str(i)))
  # splitting Dataframe (dataset not included)
  train_df = set1.iloc[train_index]
  val_df = set1.iloc[val_index]
  ds_train = Dataset.from_pandas(train_df)
  ds_validation = Dataset.from_pandas(val_df)
  tok_ds_train = ds_train.map(tok_func, batched=True, remove_columns=['text','id', 'keyword', 'location'])
  tok_ds_validation = ds_validation.map(tok_func, batched=True, remove_columns=['text','id', 'keyword', 'location'])
  ds = DatasetDict({"train":tok_ds_train, "validation":tok_ds_validation})

  # cleaning gpu and loading the model
  clean_gpu()
  model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels = 2).to(device)
  # setting up the trainer
  trainer = Trainer(model = model, args = training_args, train_dataset = ds["train"], eval_dataset = ds["validation"], compute_metrics = compute_metrics, data_collator = data_collator)
  # train the model
  trainer.train()
  # access the performance
  eval_accuracy = trainer.evaluate(ds["validation"])['eval_accuracy']
  eval_f1 = trainer.evaluate(ds["validation"])['eval_f1']
  # append model score
  f1.append(eval_f1)
  accuracy.append(eval_accuracy)

Map:   0%|          | 0/3897 [00:00<?, ? examples/s]

Map:   0%|          | 0/975 [00:00<?, ? examples/s]

BEFORE CLEANING:
Allocated: 1.00 GB
Cached: 1.34 GB




/usr/local/lib/python3.9/dist-packages/torch/cuda/memory.py:395: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


AFTER CLEANING:
Allocated: 1.00 GB
Cached: 1.08 GB


...

BEFORE CLEANING:
Allocated: 1.01 GB
Cached: 1.39 GB




/usr/local/lib/python3.9/dist-packages/torch/cuda/memory.py:395: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


AFTER CLEANING:
Allocated: 1.01 GB
Cached: 1.09 GB


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.435540,0.806982,0.806061
2,0.465700,0.418385,0.818275,0.816741
2,0.465700,0.434145,0.816222,0.814952
3,0.334800,0.438728,0.810062,0.809678


# Saving the model

In [15]:
# ===========================================
# ||                                       ||
# ||Section 11: saving the model           ||
# ||                                       ||
# ===========================================

import os
# Set the output directory
output_dir = '/content/output/DISTILBERT'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the model and tokenizer to the output directory
trainer.save_model(output_dir)
tokz.save_pretrained(output_dir)

('/content/output/DISTILBERT/tokenizer_config.json',
 '/content/output/DISTILBERT/special_tokens_map.json',
 '/content/output/DISTILBERT/vocab.txt',
 '/content/output/DISTILBERT/added_tokens.json',
 '/content/output/DISTILBERT/tokenizer.json')